In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['JAVA_HOME'] = '/Users/sb0487/Desktop/sb0487/pyspark/zulu15.32.15-ca-jdk15.0.3-macosx_aarch64/zulu-15.jdk/Contents/Home' 

In [274]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use("ggplot")
from plotly.offline import iplot, init_notebook_mode
import plotly.graph_objs as go
import cufflinks as cf
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import pandas as pd
import numpy as np
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
from ipywidgets import interact, interact_manual
from pyspark.sql.window import Window
import pendulum as plm
from pyspark.sql.types import IntegerType
from pyspark.sql import DataFrame
import pendulum
from datetime import datetime
cf.go_offline(connected=True)
init_notebook_mode(connected=True)

In [3]:
from pyspark.sql import SparkSession as Session
from pyspark import SparkConf as Conf
from pyspark.sql import functions as F, Window as W
C = F.col

In [4]:
conf = (Conf()
        .set("spark.sql.sources.partitionOverwriteMode", "dynamic")
        .set("spark.driver.memory", "3g")
        .set("spark.sql.broadcastTimeout", "10200")
        .set("spark.driver.maxResultSize", "1g")
#         .set("spark.local.dir","/buffer") 指定shuffle write的路徑
       ) 

In [5]:
spark = (Session
         .builder
         .appName("tradddddd")
         .master("local[4]")# 核心數量
         .config(conf=conf)
         .getOrCreate())

In [6]:
# UI
spark

# 說明

想法: 若期權微笑理論正確，深價外被高估，則表示過去一段時間內，買進價平的call or put 並賣出深價外，應當能獲利

回測商品: 週選

回測時間: 2018/01~2021/07

選擇權資料來源: [證交所](https://www.taifex.com.tw/cht/3/dlOptDailyMarketView)

加權指數資料來源: yfinance (yahoo 提供的 api)

作法: 每週週三買,放到隔週三結算

考量運算速度,以pyspark(平行計算框架)完成

# load data

## op data

In [233]:
import os
# 自行至證交所下載,並放到指定目錄方便讀取
path = "/Users/sb0487/Desktop/sb0487/finance/op/data/2018_to_now_opt/"
dirlist = os.listdir(path)

In [234]:
dirlist

['2018_opt_08.csv',
 '2021_opt_05.csv',
 '2021_opt_04.csv',
 '2018_opt_09.csv',
 '2021_opt_06.csv',
 '2021_opt_07.csv',
 '2021_opt_03.csv',
 '2021_opt_02.csv',
 '2021_opt_01.csv',
 '2019_opt_07.csv',
 '2019_opt_12.csv',
 '2019_opt_06.csv',
 '2020_opt_09.csv',
 '2019_opt_10.csv',
 '2019_opt_04.csv',
 '2019_opt_05.csv',
 '2019_opt_11.csv',
 '2020_opt_08.csv',
 '2019_opt_01.csv',
 '2019_opt_02.csv',
 '2019_opt_03.csv',
 '2020_opt_03.csv',
 '2020_opt_02.csv',
 '2020_opt_01.csv',
 '2020_opt_11.csv',
 '2020_opt_05.csv',
 '2019_opt_08.csv',
 '2019_opt_09.csv',
 '2020_opt_04.csv',
 '2020_opt_10.csv',
 '2020_opt_06.csv',
 '2020_opt_12.csv',
 '2020_opt_07.csv',
 '2018_opt_01.csv',
 '2018_opt_02.csv',
 '2018_opt_03.csv',
 '2018_opt_07.csv',
 '2018_opt_06.csv',
 '2018_opt_12.csv',
 '2018_opt_04.csv',
 '2018_opt_10.csv',
 '2018_opt_11.csv',
 '2018_opt_05.csv']

In [275]:
def load_op_data(path:str)->DataFrame:
    opt_sdf = spark.read.csv(path,header = True,encoding = "cp950").where(C("契約")=="TXO") #選擇台指選
    return opt_sdf
opt_sdf = load_op_data(path)

In [276]:
# 觀察欄位
opt_sdf.printSchema()

root
 |-- 交易日期: string (nullable = true)
 |-- 契約: string (nullable = true)
 |-- 到期月份(週別): string (nullable = true)
 |-- 履約價: string (nullable = true)
 |-- 買賣權: string (nullable = true)
 |-- 開盤價: string (nullable = true)
 |-- 最高價: string (nullable = true)
 |-- 最低價: string (nullable = true)
 |-- 收盤價: string (nullable = true)
 |-- 成交量: string (nullable = true)
 |-- 結算價: string (nullable = true)
 |-- 未沖銷契約數: string (nullable = true)
 |-- 最後最佳買價: string (nullable = true)
 |-- 最後最佳賣價: string (nullable = true)
 |-- 歷史最高價: string (nullable = true)
 |-- 歷史最低價: string (nullable = true)
 |-- 是否因訊息面暫停交易: string (nullable = true)
 |-- 交易時段: string (nullable = true)



In [277]:
# 資料筆數
opt_sdf.count()

851272

In [278]:
#看一下資料長相, 要有美日的加權指數資料才能知道哪邊是價平,發現這邊並沒有,因此需要自行抓取
opt_sdf.limit(5).toPandas()

,交易日期,契約,到期月份(週別),履約價,買賣權,開盤價,最高價,最低價,收盤價,成交量,結算價,未沖銷契約數,最後最佳買價,最後最佳賣價,歷史最高價,歷史最低價,是否因訊息面暫停交易,交易時段
0,2018/11/01,TXO,201811W1,8800.0000,買權,-,-,-,-,0,1010,0,-,-,-,-,None,一般
1,2018/11/01,TXO,201811W1,8800.0000,買權,-,-,-,-,0,-,-,-,-,-,-,None,盤後
2,2018/11/01,TXO,201811W1,8800.0000,賣權,2.5,2.5,1.6,2.4,3180,2.4,5093,2,2.5,7.6,1.6,None,一般
3,2018/11/01,TXO,201811W1,8800.0000,賣權,3.8,3.9,2,2.7,991,-,-,2.2,2.8,7.6,2,None,盤後
4,2018/11/01,TXO,201811W1,8900.0000,買權,-,-,-,-,0,915,0,-,-,-,-,None,一般


## 加權指數資料

In [279]:
# 透過yfinance api 拉取加權指數資料, 並轉為我們需要的spark dataframe 形式
def get_stock_price(target_stock = '^TWII',
                    start_date = datetime(2018, 1, 1),
                    end_date = datetime(2021, 7, 31))->DataFrame:
    from pandas_datareader import data
    import yfinance as yf
    yf.pdr_override() #以pandasreader常用的格式覆寫
    TWII_df = data.get_data_yahoo([target_stock], start_date, end_date) 
    TWII_df['交易日期'] = TWII_df.index.strftime("%Y/%m/%d") # formart 日期格式
    TWII_close_df = TWII_df[["Close","交易日期"]].rename(columns={"Close": "加權指數收盤價"}) 
    TWII_close_sdf = spark.createDataFrame(TWII_close_df)
    return TWII_close_sdf


In [280]:
# 打開手機檢查yfinance的資料是否正確
TWII_close_sdf = get_stock_price('^TWII',datetime(2018, 1, 1),datetime(2021, 7, 31))
TWII_close_sdf.toPandas().head(5)

[*********************100%***********************]  1 of 1 completed


,加權指數收盤價,交易日期
0,10710.730469,2018/01/02
1,10801.570312,2018/01/03
2,10848.629883,2018/01/04
3,10879.799805,2018/01/05
4,10915.750000,2018/01/08


## 按照日期將加[權指數資料]join回 [選擇權資料]

In [283]:
# 以交易日期為key, 將兩者join
def join_sdf(opt_sdf:DataFrame ,TWII_close_sdf:DataFrame,key=["交易日期"])->DataFrame:
    joined_sdf = opt_sdf.join(TWII_close_sdf,on = key).orderBy("交易日期")
    return joined_sdf
    

In [285]:
# 檢查資料是否正確
joined_sdf = join_sdf(opt_sdf,TWII_close_sdf)
joined_sdf.limit(10).toPandas()

,交易日期,契約,到期月份(週別),履約價,買賣權,開盤價,最高價,最低價,收盤價,成交量,結算價,未沖銷契約數,最後最佳買價,最後最佳賣價,歷史最高價,歷史最低價,是否因訊息面暫停交易,交易時段,加權指數收盤價
0,2018/01/02,TXO,201801W1,9800.0000,買權,-,-,-,-,0,915,0,-,-,-,-,None,一般,10710.730469
1,2018/01/02,TXO,201801W1,9700.0000,買權,-,-,-,-,0,1010,0,-,-,-,-,None,一般,10710.730469
2,2018/01/02,TXO,201801W1,9700.0000,賣權,-,-,-,-,0,-,-,-,-,-,-,None,盤後,10710.730469
3,2018/01/02,TXO,201801W1,9600.0000,賣權,-,-,-,-,0,0.1,82,-,-,-,-,None,一般,10710.730469
4,2018/01/02,TXO,201801W1,9600.0000,賣權,-,-,-,-,0,-,-,-,-,-,-,None,盤後,10710.730469
5,2018/01/02,TXO,201801W1,9700.0000,買權,-,-,-,-,0,-,-,-,-,-,-,None,盤後,10710.730469
6,2018/01/02,TXO,201801W1,9700.0000,賣權,0.3,0.3,0.3,0.3,1,0.1,199,-,0.3,0.7,0.1,None,一般,10710.730469
7,2018/01/02,TXO,201801W1,9600.0000,買權,-,-,-,-,0,1110,1,-,-,-,-,None,一般,10710.730469
8,2018/01/02,TXO,201801W1,9600.0000,買權,-,-,-,-,0,-,-,-,-,-,-,None,盤後,10710.730469
9,2018/01/02,TXO,201801W1,9800.0000,買權,-,-,-,-,0,-,-,-,-,-,-,None,盤後,10710.730469


# 資料預處理

有三個項目要做

1.因策略是根據固定檔數的價外做買賣,EX:買進價外3檔,放空價外6檔,因此必須與加權指數收盤價做比對,才能知道當前價平合約之履約價

2.將日期轉成the day of week

3.抓出星期四那天的淺價外與深價外的履約價

## 抓出價平

1. 將加權指數收盤價四捨五入合併至其最接近的履約價,ex:12100->12100; 12130->12150
2. 新增一欄位"at_the_money",若該履約價為價平,則"是",otherwise"否"

In [294]:
def get_atm_sdf(sdf:DataFrame)->DataFrame:
    atm_sdf = (
        sdf
        .where(C("交易時段")=="一般") #選擇交易時間"一般"
        .where(C("到期月份(週別)").contains("W")) #選擇週選
        .withColumn("加權指數收盤價_餘數",C("加權指數收盤價")%50)
        .withColumn("加權指數收盤價_扣除餘數加50",C("加權指數收盤價")-C("加權指數收盤價_餘數")+50)
        .withColumn("加權指數收盤價_四捨五入",F.when(C("加權指數收盤價_餘數")>25,C("加權指數收盤價_扣除餘數加50")).otherwise(C("加權指數收盤價_扣除餘數加50")-50))
        .drop("加權指數收盤價_餘數","加權指數收盤價_扣除餘數加50")
        .withColumn("at_the_money",F.when(C("履約價")==C("加權指數收盤價_四捨五入"),"是").otherwise("否"))
        .withColumn("履約價與價平價差",(C("履約價")-C("加權指數收盤價_四捨五入")))
    )
    return atm_sdf


In [289]:
atm_sdf = get_atm_sdf(joined_sdf)

In [290]:
#檢查不是價平者有無錯誤
atm_sdf.where(C("at_the_money")=="否").limit(10).toPandas()

,交易日期,契約,到期月份(週別),履約價,買賣權,開盤價,最高價,最低價,收盤價,成交量,...,最後最佳買價,最後最佳賣價,歷史最高價,歷史最低價,是否因訊息面暫停交易,交易時段,加權指數收盤價,加權指數收盤價_四捨五入,at_the_money,履約價與價平價差
0,2018/01/02,TXO,201801W1,10000.0000,買權,-,-,-,-,0,...,-,-,-,-,None,一般,10710.730469,10700.0,否,-700.0
1,2018/01/02,TXO,201801W1,9800.0000,買權,-,-,-,-,0,...,-,-,-,-,None,一般,10710.730469,10700.0,否,-900.0
2,2018/01/02,TXO,201801W1,9900.0000,賣權,-,-,-,-,0,...,-,-,-,-,None,一般,10710.730469,10700.0,否,-800.0
3,2018/01/02,TXO,201801W1,9700.0000,買權,-,-,-,-,0,...,-,-,-,-,None,一般,10710.730469,10700.0,否,-1000.0
4,2018/01/02,TXO,201801W1,9700.0000,賣權,0.3,0.3,0.3,0.3,1,...,-,0.3,0.7,0.1,None,一般,10710.730469,10700.0,否,-1000.0
5,2018/01/02,TXO,201801W1,9800.0000,賣權,-,-,-,-,0,...,-,-,-,-,None,一般,10710.730469,10700.0,否,-900.0
6,2018/01/02,TXO,201801W1,9900.0000,買權,-,-,-,-,0,...,-,-,-,-,None,一般,10710.730469,10700.0,否,-800.0
7,2018/01/02,TXO,201801W1,9600.0000,買權,-,-,-,-,0,...,-,-,-,-,None,一般,10710.730469,10700.0,否,-1100.0
8,2018/01/02,TXO,201801W1,9600.0000,賣權,-,-,-,-,0,...,-,-,-,-,None,一般,10710.730469,10700.0,否,-1100.0
9,2018/01/02,TXO,201801W1,10000.0000,賣權,0.2,0.2,0.2,0.2,65,...,-,0.3,7,0.2,None,一般,10710.730469,10700.0,否,-700.0


In [291]:
#檢查是價平者有無錯誤
atm_sdf.where(C("at_the_money")=="是").limit(5).toPandas()

,交易日期,契約,到期月份(週別),履約價,買賣權,開盤價,最高價,最低價,收盤價,成交量,...,最後最佳買價,最後最佳賣價,歷史最高價,歷史最低價,是否因訊息面暫停交易,交易時段,加權指數收盤價,加權指數收盤價_四捨五入,at_the_money,履約價與價平價差
0,2018/01/02,TXO,201801W1,10700.0000,賣權,78,78,14,15,49803,...,14.5,15,234,14,None,一般,10710.730469,10700.0,是,0.0
1,2018/01/02,TXO,201801W1,10700.0000,買權,6.1,32,6,31.5,55870,...,31,32,32,0.7,None,一般,10710.730469,10700.0,是,0.0
2,2018/01/03,TXO,201801W2,10800.0000,買權,28,58,28,53,11248,...,52,53,58,28,None,一般,10801.570312,10800.0,是,0.0
3,2018/01/03,TXO,201801W1,10800.0000,買權,2.9,14.5,0.1,0.1,121925,...,-,0.1,14.5,0.1,None,一般,10801.570312,10800.0,是,0.0
4,2018/01/03,TXO,201801W2,10800.0000,賣權,90,90,52,54,6089,...,54,55,90,52,None,一般,10801.570312,10800.0,是,0.0


## 將日期轉成the day of week

In [301]:
# 包成udf
def get_day_of_week(date):
    dt = pendulum.parse(date)
    return dt.day_of_week
func = F.udf(get_day_of_week)

def get_day_of_week_sdf(sdf:DataFrame)->DataFrame:
    day_of_week_sdf = (
        sdf
        .withColumn("day_of_week",func(C("交易日期")))
    )
    return day_of_week_sdf

In [299]:
day_of_week_sdf = get_day_of_week_sdf(atm_sdf)

In [300]:
# 檢查日期轉換有無錯誤
day_of_week_sdf.limit(100).toPandas()

,交易日期,契約,到期月份(週別),履約價,買賣權,開盤價,最高價,最低價,收盤價,成交量,...,最後最佳賣價,歷史最高價,歷史最低價,是否因訊息面暫停交易,交易時段,加權指數收盤價,加權指數收盤價_四捨五入,at_the_money,履約價與價平價差,day_of_week
0,2018/01/02,TXO,201801W1,9600.0000,買權,-,-,-,-,0,...,-,-,-,None,一般,10710.730469,10700.0,否,-1100.0,2
1,2018/01/02,TXO,201801W1,9600.0000,賣權,-,-,-,-,0,...,-,-,-,None,一般,10710.730469,10700.0,否,-1100.0,2
2,2018/01/02,TXO,201801W1,9700.0000,買權,-,-,-,-,0,...,-,-,-,None,一般,10710.730469,10700.0,否,-1000.0,2
3,2018/01/02,TXO,201801W1,9700.0000,賣權,0.3,0.3,0.3,0.3,1,...,0.3,0.7,0.1,None,一般,10710.730469,10700.0,否,-1000.0,2
4,2018/01/02,TXO,201801W1,9800.0000,買權,-,-,-,-,0,...,-,-,-,None,一般,10710.730469,10700.0,否,-900.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2018/01/03,TXO,201801W1,10800.0000,賣權,59,62,5.3,8,109001,...,8.1,333,5.3,None,一般,10801.570312,10800.0,是,0.0,3
96,2018/01/03,TXO,201801W1,10850.0000,買權,0.4,1.6,0.1,0.1,13916,...,0.2,1.6,0.1,None,一般,10801.570312,10800.0,否,50.0,3
97,2018/01/03,TXO,201801W1,10850.0000,賣權,105,108,48,58,20323,...,59,264,48,None,一般,10801.570312,10800.0,否,50.0,3
98,2018/01/03,TXO,201801W1,10900.0000,買權,0.3,0.4,0.1,0.1,3978,...,0.2,0.5,0.1,None,一般,10801.570312,10800.0,否,100.0,3


## 選定要操作的兩檔履約價

以星期三開倉當天的收盤價找出該週要操作的賣賣權賣權

1.價外1檔(淺價外,deep_oftm)

2.價外6檔這兩個商品(深價外,shallow_oftm)

之履約價商品

oftm = out of the mony

In [462]:
#透過deep_oftm_point與shallow_oftm_point這兩個參數決定價外幾檔
#預設以價平加上300點作為深價外,價平家上100點作為淺價外

def select_strike_price(sdf:DataFrame,deep_oftm_point = 250,shallow_oftm_point = 150)->DataFrame:
    overCategory = Window.partitionBy("到期月份(週別)").orderBy(F.desc("到期月份(週別)"))


    strike_price_sdf = (
        sdf
    #     .where((C("day_of_week")=="3")|(C("day_of_week")=="5"))
        .where(C("day_of_week")=="3")
        .withColumn("該週3開倉_加權指數收盤價_四捨五入",F.first("加權指數收盤價_四捨五入").over(overCategory))
        .orderBy("交易日期","到期月份(週別)")
        .withColumn("put_strike_price",
                F.when((C("履約價")==C("該週3開倉_加權指數收盤價_四捨五入")-deep_oftm_point)&(C("買賣權")=="賣權"),"put_deep_oftm")
                .when((C("履約價")==C("該週3開倉_加權指數收盤價_四捨五入")-shallow_oftm_point)&(C("買賣權")=="賣權"),"put_shallow_oftm")
               )

        .withColumn("call_strike_price",
                F.when((C("履約價")==C("該週3開倉_加權指數收盤價_四捨五入")+deep_oftm_point)&(C("買賣權")=="買權"),"call_deep_oftm")
                .when((C("履約價")==C("該週3開倉_加權指數收盤價_四捨五入")+shallow_oftm_point)&(C("買賣權")=="買權"),"call_shallow_oftm")
               )
        .drop("該週3開倉_加權指數收盤價_四捨五入","該週3開倉_加權指數收盤價_四捨五入")
        .where(
            (C("put_strike_price").isNotNull())
            |
            (C("call_strike_price").isNotNull())


        )


    )
    return strike_price_sdf

In [463]:
strike_price_sdf = select_strike_price(day_of_week_sdf,300,50)
strike_price_df = strike_price_sdf.toPandas()

In [359]:
"""
再次檢查
1.該日期的加權指數收盤價是否正確
2.價內價外判斷是否正確
3.星期幾的轉換是否正確
"""

strike_price_df.iloc[0:50]

,交易日期,契約,到期月份(週別),履約價,買賣權,開盤價,最高價,最低價,收盤價,成交量,...,歷史最低價,是否因訊息面暫停交易,交易時段,加權指數收盤價,加權指數收盤價_四捨五入,at_the_money,履約價與價平價差,day_of_week,put_strike_price,call_strike_price
0,2018/01/03,TXO,201801W1,10500.0000,賣權,0.3,0.4,0.1,0.1,3772,...,0.1,None,一般,10801.570312,10800.0,否,-300.0,3,put_deep_oftm,None
1,2018/01/03,TXO,201801W1,10750.0000,賣權,18.5,24,0.1,0.1,84171,...,0.1,None,一般,10801.570312,10800.0,否,-50.0,3,put_shallow_oftm,None
2,2018/01/03,TXO,201801W1,10850.0000,買權,0.4,1.6,0.1,0.1,13916,...,0.1,None,一般,10801.570312,10800.0,否,50.0,3,None,call_shallow_oftm
3,2018/01/03,TXO,201801W1,11100.0000,買權,0.1,0.1,0.1,0.1,20,...,0.1,None,一般,10801.570312,10800.0,否,300.0,3,None,call_deep_oftm
4,2018/01/03,TXO,201801W2,10500.0000,賣權,4,5.8,3.3,3.4,5526,...,3.3,None,一般,10801.570312,10800.0,否,-300.0,3,put_deep_oftm,None
5,2018/01/03,TXO,201801W2,10750.0000,賣權,54,83,33.5,34.5,12063,...,33.5,None,一般,10801.570312,10800.0,否,-50.0,3,put_shallow_oftm,None
6,2018/01/03,TXO,201801W2,10850.0000,買權,16,34.5,14,30,11188,...,14,None,一般,10801.570312,10800.0,否,50.0,3,None,call_shallow_oftm
7,2018/01/03,TXO,201801W2,11100.0000,買權,0.6,1,0.6,0.7,2608,...,0.6,None,一般,10801.570312,10800.0,否,300.0,3,None,call_deep_oftm
8,2018/01/10,TXO,201801W2,10500.0000,賣權,0.1,0.1,0.1,0.1,150,...,0.1,None,一般,10831.089844,10850.0,否,-350.0,3,put_deep_oftm,None
9,2018/01/10,TXO,201801W2,10750.0000,賣權,0.3,1.1,0.1,0.1,11503,...,0.1,None,一般,10831.089844,10850.0,否,-100.0,3,put_shallow_oftm,None


# 策略與結果

## 策略1,價差

每週五買進價外2檔(100點)與賣出價外6檔(300點)

放到下週三做結算

In [464]:

def execution_strategy(sdf,買賣權 = "賣權",fee:"point"=1,買進賣遠 = True)->DataFrame:
    overCategory = Window.partitionBy(["到期月份(週別)","put_strike_price","call_strike_price"]).orderBy(F.desc("到期月份(週別)"))
    if 買進賣遠:
        sign = -1
    else:
        sign = 1
    if 買賣權 == "賣權":
        result_sdf = (
            sdf
        #     .where(C("買賣權")=="賣權")
            .withColumn("put_profit_deep",
                        F.when(C("put_strike_price")=="put_deep_oftm",(C("收盤價")-F.lag("收盤價").over(overCategory))*sign-fee).otherwise(0)
                       )
            .withColumn("put_profit_shallow",
                        F.when(C("put_strike_price")=="put_shallow_oftm",(-C("收盤價")+F.lag("收盤價").over(overCategory))*sign-fee).otherwise(0)
                       )
            .withColumn("total_profit",C("put_profit_deep")+C("put_profit_shallow"))
        )
    if 買賣權 == "買權":
        result_sdf = (
            sdf
        #     .where(C("買賣權")=="賣權")
            .withColumn("call_profit_deep",
                        F.when(C("call_strike_price")=="call_deep_oftm",(C("收盤價")-F.lag("收盤價").over(overCategory))*sign-fee).otherwise(0)
                       )
            .withColumn("call_profit_shallow",
                        F.when(C("call_strike_price")=="call_shallow_oftm",(-C("收盤價")+F.lag("收盤價").over(overCategory))*sign-fee).otherwise(0)
                       )
            .withColumn("total_profit",C("call_profit_deep")+C("call_profit_shallow"))
        )

    result_sdf=(

        result_sdf
        .withColumn("total_profit",F.when(C("total_profit").isNotNull(),C("total_profit")).otherwise(0))
        .orderBy("交易日期","到期月份(週別)")

    )
        
        
        
           
    return result_sdf 

In [503]:
result_sdf = execution_strategy(strike_price_sdf,"賣權",1,True)

In [504]:
result_df = result_sdf.toPandas()

In [505]:
result_df.iloc[0:50]

,交易日期,契約,到期月份(週別),履約價,買賣權,開盤價,最高價,最低價,收盤價,成交量,...,加權指數收盤價,加權指數收盤價_四捨五入,at_the_money,履約價與價平價差,day_of_week,put_strike_price,call_strike_price,put_profit_deep,put_profit_shallow,total_profit
0,2018/01/03,TXO,201801W1,10650.0000,賣權,1,1.9,0.1,0.1,20595,...,10801.570312,10800.0,否,-150.0,3,put_shallow_oftm,None,0.0,NaN,0.0
1,2018/01/03,TXO,201801W1,10550.0000,賣權,0.3,0.4,0.1,0.1,3654,...,10801.570312,10800.0,否,-250.0,3,put_deep_oftm,None,NaN,0.0,0.0
2,2018/01/03,TXO,201801W1,10950.0000,買權,0.1,0.1,0.1,0.1,360,...,10801.570312,10800.0,否,150.0,3,None,call_shallow_oftm,0.0,0.0,0.0
3,2018/01/03,TXO,201801W2,10950.0000,買權,3.5,9.1,2.8,7.1,5270,...,10801.570312,10800.0,否,150.0,3,None,call_shallow_oftm,0.0,0.0,0.0
4,2018/01/03,TXO,201801W2,10550.0000,賣權,7,9,4.6,4.8,10315,...,10801.570312,10800.0,否,-250.0,3,put_deep_oftm,None,NaN,0.0,0.0
5,2018/01/03,TXO,201801W2,10650.0000,賣權,26,26,12.5,13,7241,...,10801.570312,10800.0,否,-150.0,3,put_shallow_oftm,None,0.0,NaN,0.0
6,2018/01/10,TXO,201801W2,10950.0000,買權,3.3,5.8,0.1,0.2,43050,...,10831.089844,10850.0,否,100.0,3,None,call_shallow_oftm,0.0,0.0,0.0
7,2018/01/10,TXO,201801W2,11050.0000,買權,0.2,0.2,0.1,0.1,752,...,10831.089844,10850.0,否,200.0,3,None,call_deep_oftm,0.0,0.0,0.0
8,2018/01/10,TXO,201801W2,10550.0000,賣權,0.2,0.2,0.1,0.1,795,...,10831.089844,10850.0,否,-300.0,3,put_deep_oftm,None,3.7,0.0,3.7
9,2018/01/10,TXO,201801W2,10650.0000,賣權,0.2,0.3,0.1,0.1,991,...,10831.089844,10850.0,否,-200.0,3,put_shallow_oftm,None,0.0,-13.9,-13.9


In [506]:
def visualize(sdf):
    #取出獲利欄位
    profit_sdf = (
        sdf
        .groupBy("交易日期")
        .agg(F.sum("total_profit").alias("total_profit"))
        .orderBy("交易日期")
    )
    profit_df = profit_sdf.toPandas()
    profit_df["sum_profit"] = profit_df["total_profit"].cumsum()
    #繪圖
    layout = go.Layout(title='累計獲利圖', xaxis=dict(title='Date'), yaxis=dict(title='Point'))


    
    data = [
    {
        'x': profit_df.交易日期,    
        'y': profit_df.sum_profit,
        
    }
        
    
    ]
    
    join_df5.iplot(data = data, layout = layout)
    

In [507]:
visualize(result_sdf,)

In [521]:
"""
策略測試
"""
strike_price_sdf = select_strike_price(day_of_week_sdf,300,100)
result_sdf = execution_strategy(strike_price_sdf,"賣權",1,False)
visualize(result_sdf,)

In [511]:
result_df = result_sdf.toPandas()

In [513]:
result_df.iloc[0:50]

,交易日期,契約,到期月份(週別),履約價,買賣權,開盤價,最高價,最低價,收盤價,成交量,...,加權指數收盤價,加權指數收盤價_四捨五入,at_the_money,履約價與價平價差,day_of_week,put_strike_price,call_strike_price,call_profit_deep,call_profit_shallow,total_profit
0,2018/01/03,TXO,201801W1,10750.0000,賣權,18.5,24,0.1,0.1,84171,...,10801.570312,10800.0,否,-50.0,3,put_shallow_oftm,None,0.0,0.0,0.0
1,2018/01/03,TXO,201801W1,11100.0000,買權,0.1,0.1,0.1,0.1,20,...,10801.570312,10800.0,否,300.0,3,None,call_deep_oftm,NaN,0.0,0.0
2,2018/01/03,TXO,201801W1,10500.0000,賣權,0.3,0.4,0.1,0.1,3772,...,10801.570312,10800.0,否,-300.0,3,put_deep_oftm,None,0.0,0.0,0.0
3,2018/01/03,TXO,201801W1,10850.0000,買權,0.4,1.6,0.1,0.1,13916,...,10801.570312,10800.0,否,50.0,3,None,call_shallow_oftm,0.0,NaN,0.0
4,2018/01/03,TXO,201801W2,10850.0000,買權,16,34.5,14,30,11188,...,10801.570312,10800.0,否,50.0,3,None,call_shallow_oftm,0.0,NaN,0.0
5,2018/01/03,TXO,201801W2,11100.0000,買權,0.6,1,0.6,0.7,2608,...,10801.570312,10800.0,否,300.0,3,None,call_deep_oftm,NaN,0.0,0.0
6,2018/01/03,TXO,201801W2,10500.0000,賣權,4,5.8,3.3,3.4,5526,...,10801.570312,10800.0,否,-300.0,3,put_deep_oftm,None,0.0,0.0,0.0
7,2018/01/03,TXO,201801W2,10750.0000,賣權,54,83,33.5,34.5,12063,...,10801.570312,10800.0,否,-50.0,3,put_shallow_oftm,None,0.0,0.0,0.0
8,2018/01/10,TXO,201801W2,10850.0000,買權,56,76,0.1,0.1,135602,...,10831.089844,10850.0,是,0.0,3,None,call_shallow_oftm,0.0,28.9,28.9
9,2018/01/10,TXO,201801W2,11100.0000,買權,0.1,0.1,0.1,0.1,254,...,10831.089844,10850.0,否,250.0,3,None,call_deep_oftm,-1.6,0.0,-1.6


# 延伸策略

## 單向buy

In [528]:
def select_strike_price(sdf:DataFrame,oftm_point = 250)->DataFrame:
    overCategory = Window.partitionBy("到期月份(週別)").orderBy(F.desc("到期月份(週別)"))


    strike_price_sdf = (
        sdf
    #     .where((C("day_of_week")=="3")|(C("day_of_week")=="5"))
        .where(C("day_of_week")=="3")
        .withColumn("該週3開倉_加權指數收盤價_四捨五入",F.first("加權指數收盤價_四捨五入").over(overCategory))
        .orderBy("交易日期","到期月份(週別)")
        .withColumn("put_strike_price",
                F.when((C("履約價")==C("該週3開倉_加權指數收盤價_四捨五入")-oftm_point)&(C("買賣權")=="賣權"),"put_oftm_point")
               )

        .withColumn("call_strike_price",
                F.when((C("履約價")==C("該週3開倉_加權指數收盤價_四捨五入")+oftm_point)&(C("買賣權")=="買權"),"call_oftm_point")
               )
        .drop("該週3開倉_加權指數收盤價_四捨五入","該週3開倉_加權指數收盤價_四捨五入")
        .where(
            (C("put_strike_price").isNotNull())
            |
            (C("call_strike_price").isNotNull())


        )


    )
    return strike_price_sdf

In [541]:

def execution_strategy(sdf,買賣權 = "賣權",fee:"point"=1,buy = True)->DataFrame:
    overCategory = Window.partitionBy(["到期月份(週別)","put_strike_price","call_strike_price"]).orderBy(F.desc("到期月份(週別)"))
    if buy:
        sign = 1
    else:
        sign = -1
    if 買賣權 == "賣權":
        result_sdf = (
            sdf
        #     .where(C("買賣權")=="賣權")
            .withColumn("total_profit",
                        F.when(C("put_strike_price")=="put_oftm_point",(C("收盤價")-F.lag("收盤價").over(overCategory))*sign-fee).otherwise(0)
                       )
#             .withColumn("put_profit_shallow",
#                         F.when(C("put_strike_price")=="put_shallow_oftm",(-C("收盤價")+F.lag("收盤價").over(overCategory))*sign-fee).otherwise(0)
#                        )
#             .withColumn("total_profit",C("put_profit_deep")+C("put_profit_shallow"))
        )
    if 買賣權 == "買權":
        result_sdf = (
            sdf
        #     .where(C("買賣權")=="賣權")
            .withColumn("total_profit",
                        F.when(C("call_strike_price")=="call_oftm_point",(C("收盤價")-F.lag("收盤價").over(overCategory))*sign-fee).otherwise(0)
                       )
#             .withColumn("call_profit_shallow",
#                         F.when(C("call_strike_price")=="call_shallow_oftm",(-C("收盤價")+F.lag("收盤價").over(overCategory))*sign-fee).otherwise(0)
#                        )
#             .withColumn("total_profit",C("call_profit_deep")+C("call_profit_shallow"))
        )

    result_sdf=(

        result_sdf
        .withColumn("total_profit",F.when(C("total_profit").isNotNull(),C("total_profit")).otherwise(0))
        .orderBy("交易日期","到期月份(週別)")

    )
        
        
        
           
    return result_sdf 

In [560]:
strike_price_sdf = select_strike_price(day_of_week_sdf,100)
result_sdf = execution_strategy(strike_price_sdf,"賣權",1,buy = False)
visualize(result_sdf,)